In [2]:
import os
from dotmap import DotMap
import torch
from utils import video_transforms

In [3]:
arg={'num_seg':32,
    'length':32,
    'scale':0.5,
    'settings':'/home/thanhdang/projects/vinbrain_internship/action_recognition/datasets/settings',
    'dataset':'rwf_2000',
    'root':'/home/thanhdang/datasets/RWF-2000/rawframes'}
arg=DotMap(arg)
modality='rgb'
is_color=True

In [4]:
input_size = int(224 * arg.scale)
width = int(340 * arg.scale)
height = int(256 * arg.scale)
scale_ratios = [1.0, 0.875, 0.75, 0.66]


In [5]:
clip_mean = [0.43216, 0.394666, 0.37645] * arg.num_seg * arg.length
clip_std = [0.22803, 0.22145, 0.216989] * arg.num_seg * arg.length

In [6]:
normalize = video_transforms.Normalize(mean=clip_mean,std=clip_std)
train_transform = video_transforms.Compose([
                  video_transforms.MultiScaleCrop((input_size, input_size), scale_ratios),
                  video_transforms.RandomHorizontalFlip(),
                  video_transforms.ToTensor(),
                  normalize,
            ])
    
val_transform = video_transforms.Compose([
                video_transforms.CenterCrop((input_size)),
                video_transforms.ToTensor(),
                normalize,
            ])

In [7]:
train_setting_file = "train_%s.txt" % (modality)
train_split_file = os.path.join(arg.settings, arg.dataset, train_setting_file)
val_setting_file = "validation_%s.txt" % (modality)
val_split_file = os.path.join(arg.settings, arg.dataset, val_setting_file)

In [8]:
import datasets

train_dataset = datasets.__dict__[arg.dataset](root=arg.root,
                                            source=train_split_file,
                                            phase="train",
                                            modality=modality,
                                            is_color=is_color,
                                            new_length=arg.length,
                                            new_width=width,
                                            new_height=height,
                                            video_transform=train_transform,
                                            num_segments=arg.num_seg)

val_dataset = datasets.__dict__[arg.dataset](root=arg.root,
                                          source=val_split_file,
                                          phase="val",
                                          modality=modality,
                                          is_color=is_color,
                                          new_length=arg.length,
                                          new_width=width,
                                          new_height=height,
                                          video_transform=val_transform,
                                          num_segments=arg.num_seg)

In [9]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=2,
                                               pin_memory=True,
                                               num_workers=2)
from tqdm import tqdm
pbar = tqdm(train_dataloader, ncols=80, desc='Training')
for step, minibatch in enumerate(pbar):
    print(minibatch[0].shape)

Training:   0%|                               | 1/639 [00:28<5:08:06, 28.98s/it]torch.Size([2, 3072, 112, 112])
torch.Size([2, 3072, 112, 112])
Training:   0%|▏                              | 3/639 [00:38<1:56:10, 10.96s/it]torch.Size([2, 3072, 112, 112])
torch.Size([2, 3072, 112, 112])
Training:   1%|▏                              | 5/639 [00:56<1:45:07,  9.95s/it]torch.Size([2, 3072, 112, 112])
torch.Size([2, 3072, 112, 112])
Could not load file /home/thanhdang/datasets/RWF-2000/rawframes/_urlgot_121/img_00002.jpg
Training:   1%|▎                              | 6/639 [01:08<2:00:39, 11.44s/it]


RuntimeError: DataLoader worker (pid(s) 129472) exited unexpectedly

# Data handler

In [19]:
from datasets.data_handler import DataHandler
scale=0.5

In [20]:
input_size = int(224 * scale)
width = int(340 * scale)
height = int(256 * scale)
scale_ratios = [1.0, 0.875, 0.75, 0.66]
train_dataset_config = {
    'root':'/home/thanhdang/datasets/RWF-2000_v2/RWF-2000-raw-frames',
    'source':'/home/thanhdang/projects/vinbrain_internship/action_recognition/datasets/settings/rwf_2000/train_rgb.txt',
    'phase':"train",
    'modality':'rgb',
    'is_color':True,
    'new_length':32,
    'new_width':width,
    'new_height':height,
    'video_transform':train_transform,
    'num_segments':32
}
test_dataset_config = {
    'root':'/home/thanhdang/datasets/RWF-2000_v2/RWF-2000-raw-frames',
    'source':'/home/thanhdang/projects/vinbrain_internship/action_recognition/datasets/settings/rwf_2000/train_rgb.txt',
    'phase':"val",
    'modality':'rgb',
    'is_color':True,
    'new_length':32,
    'new_width':width,
    'new_height':height,
    'video_transform':val_transform,
    'num_segments':32
}
dl_cfgs = {'bs': 2, 'n_workers': 2}

NameError: name 'train_transform' is not defined